**Imports**

In [51]:
!pip install validate-docbr
!pip install pyarrow


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 25.3 MB/s eta 0:00:00m eta 0:00:010:00:01


In [52]:
import pyarrow.parquet as pq
import pandas as pd
from validate_docbr import CPF, CNPJ
import re

**Dados descritivos**

In [36]:
base_cadastral = pd.read_csv("dados_cadastrais_fake.csv", sep = ';')

In [3]:
base_cadastral.head()

,nomes,idade,cidade,estado,cpf,cnpj
0,Dennis Daniels,31,ACRELÂNDIA,AC,97566536800,06589184909526
1,Leah Becker,42,ÁGUA BRANCA,AL,425.263.807-07,25.673.336/2350-20
2,Sally Ford,18,ALVARÃES,AM,34647754103,26543101702989
3,Colleen Duncan,21,SERRA DO NAVIO,AP,252.531.560-03,19.062.080/5100-98
4,Jeff Stephenson,73,ABAÍRA,BA,49668886542,97794530015384


In [4]:
base_cadastral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nomes   10000 non-null  object
 1   idade   10000 non-null  int64 
 2   cidade  10000 non-null  object
 3   estado  10000 non-null  object
 4   cpf     10000 non-null  object
 5   cnpj    10000 non-null  object
dtypes: int64(1), object(5)
memory usage: 468.9+ KB


**Normalizando os dados**

In [5]:
#Função que normaliza a documentação
def normaliza_documento(coluna):
    """
    Remove caracteres indesejáveis dos campos selecionados    
    
    :param coluna: str. Nome da coluna a ser aplicada a lógica de remoção
        
    """
    
    
    dado_modificado = coluna.str.replace('[ .\-/]', '', regex=True)
    
    return dado_modificado
    

In [37]:
base_cadastral[['cpf', 'cnpj']] = base_cadastral[['cpf', 'cnpj']].apply(normaliza_documento)

In [38]:
base_cadastral[['estado','cpf', 'cnpj']] = base_cadastral[['estado','cpf', 'cnpj']].astype('string')


In [8]:
#chamando as funções importadas
check_cpf = CPF()

check_cnpj = CNPJ()

**Regras Validação**

In [9]:
def valida_documento(documento):
    """
    Valida se a string de parâmetro corresponde ao padrão de documentação brasileiro   
    
    :param documento: str. Numero documento analisado 
        
    """
    
    documento = str(documento)
    
    if len(documento) == 11:
        if check_cpf.validate(documento) is True:
            status = 'Válido'
             
        else:
            status = 'Inválido'
                    
    
    elif len(documento) == 14:
        if check_cnpj.validate(documento) is True:
            status = 'Válido'
            
        else:
            status = 'Inválido'
    else:
        status = 'Inválido'
            
    return status
    

In [39]:
base_cadastral['cpf_status'] = base_cadastral['cpf'].apply(valida_documento)
base_cadastral['cnpj_status'] = base_cadastral['cnpj'].apply(valida_documento)

In [22]:
base_cadastral.head()


,nomes,idade,cidade,estado,cpf,cnpj,cpf_status,cnpj_status
0,Dennis Daniels,31,ACRELÂNDIA,AC,97566536800,06589184909526,Válido,Válido
1,Leah Becker,42,ÁGUA BRANCA,AL,42526380707,25673336235020,Válido,Válido
2,Sally Ford,18,ALVARÃES,AM,34647754103,26543101702989,Válido,Válido
3,Colleen Duncan,21,SERRA DO NAVIO,AP,25253156003,19062080510098,Válido,Válido
4,Jeff Stephenson,73,ABAÍRA,BA,49668886542,97794530015384,Válido,Válido


**Quantos clientes temos na base?**

In [12]:
base_cadastral['nomes'].count()

10000

**Qual a média de idade do cliente**

In [13]:
base_cadastral['idade'].mean()

53.7831

**Quantos clientes nessa base pertencem a cada estado?**

In [44]:
base_cadastral.groupby('estado')[['nomes']].count()

,nomes
estado,
AC,371
AL,371
AM,371
AP,371
BA,371
CE,371
DF,371
ES,371
GO,371


In [40]:
#csv continha espaços entre nomes
base_cadastral['estado']  = base_cadastral['estado'].replace(
{"rio de  janeiro": "RJ", "sao  paulo": "SP","são  paulo": "SP"}, regex=True)

In [41]:
base_cadastral['estado'] = base_cadastral['estado'].replace(
    {'MINAS GERAI': 'MG','MINAS GERAIs': 'MG', 'MGs': 'MG','distrito federal': 'DF'}, regex=True)

In [42]:
base_cadastral['estado']  = base_cadastral['estado'].replace(' ','')

In [43]:
base_cadastral.groupby('estado')[['nomes']].count()

,nomes
estado,
AC,371
AL,371
AM,371
AP,371
BA,371
CE,371
DF,371
ES,371
GO,371


**Quantos CPFs/CNPJs válidos e inválidos foram encontrados??**

In [45]:
base_cadastral.groupby('cpf_status',)[['nomes']].count()

,nomes
cpf_status,
Válido,10000


In [46]:
base_cadastral.groupby('cnpj_status',)[['nomes']].count()

,nomes
cnpj_status,
Válido,10000


**Escrita de saída**

In [48]:
base_cadastral.to_csv('base_cadastral_limpa.csv',index=False)

In [53]:
base_cadastral.to_parquet('base_cadastral_limpa.parquet.gzip',compression='gzip')